In [1]:
import json
import pandas as pd

In [2]:
# https://apollo.nvu.vsc.edu/classes/remote/lecture_notes/radar/88d/88D_locations.html

# Load the HTML tables into a list of DataFrames
dfs = pd.read_html('blob/88d-locations.html', header=0)
df = dfs[0]

# Rename the columns for easier access
df.columns = ['wban', 'site', 'name', 'coordinate', 'elevation', 'height']

print(df)

      wban  site                name        coordinate  elevation  height
0    14929  KABR        Aberdeen, SD  452721 / 0982447       1302      20
1    54766  KENX          Albany, NY  423511 / 0740350       1826      20
2    03019  KABX     Albuquerque, NM  350859 / 1064926       5870      20
3    03981  KFDR       Altus AFB, OK  342144 / 0985835       1267      10
4    23047  KAMA        Amarillo, TX  351400 / 1014233       3587      20
..     ...   ...                 ...               ...        ...     ...
153  03995  KVNX       Vance AFB, OK  364427 / 0980740       1210      10
154  93234  KVBX  Vandenberg AFB, CA  345017 / 1202349       1223      20
155  03928  KICT         Wichita, KS  373917 / 0972634       1335      10
156  93774  KLTX      Wilmington, NC  335922 / 0782544         64      20
157  53116  KYUX            Yuma, AZ  322943 / 1143924        174      10

[158 rows x 6 columns]


In [3]:
# Raw location in deg/min/sec
def parse_coord_str(string):
    return float(string[:-4]) + float(string[-4:-2]) / 60.0 + float(string[-2:]) / 3600.0

# Raw elevation in feet
def ft2m(ft):
    return ft * 0.3048

# WBAN may be "PENDING"
def parse_int(string):
    try:
        i = int(string)
    except:
        i = None
    return i

In [4]:
df.set_index('site', inplace=True)
collection = df.to_dict(orient='index')

In [5]:
new_collection = {}
for key, row in collection.items():
    lat_lon_str = row['coordinate'].split('/')
    lat = parse_coord_str(lat_lon_str[0].strip())
    lon_str = lat_lon_str[1].strip()
    if lon_str[-1] == 'E':
        lon = parse_coord_str(lon_str[:-1])
    else:
        lon = -parse_coord_str(lon_str)
    elevation = ft2m(float(row['elevation']))
    wban = parse_int(row['wban'])
    new_collection[key] = {'latitude': lat, 'longitude': lon, 'elevation': elevation, 'height':float(row['height']), 'wban': wban}

In [ ]:
with open('blob/nexrad-locations.json', 'w') as fid:
    fid.write(json.dumps(new_collection, indent=4))